Notebook uses BedMachine Antarctica ice-surface and bed topographies to calculate and export subglacial hydropotential using the Shreve (1972) hydropotential equation.

Written 2025/07/19 by W. Sauthoff (wsauthoff.github.io) and M. R. Siegfried (mrsiegfried.github.io).

# Setup computing environment

In [ ]:
# Import packages
import earthaccess
import fsspec
import json
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import os
from pyproj import CRS
import shutil
import xarray as xr
import zarr
import zipfile

os.makedirs('output', exist_ok = True)

%matplotlib widget

# Access data

In [ ]:
# Direct-access stream BedMachine Antarctica bed and surface topography v3 (Morlighem et al., 2020; Morlighem, 2022)
# https://nsidc.org/data/NSIDC-0756
try:
    results = earthaccess.search_data(
        doi='10.5067/FPSU0V1MWUB6',
        cloud_hosted=True,
        # bounding_box=(1, -89, -1, -89)
    )
    
    if not results:
        raise ValueError('No datasets found matching the search criteria')
        
    # Open data granules as s3 files to stream
    files = earthaccess.open(results)
    
    # Check if files list is empty
    if not files:
        raise FileNotFoundError('No files were opened from the search results')
        
    # Check if first file exists/is valid
    if isinstance(files[0], FileNotFoundError):
        raise FileNotFoundError(f'Could not access file: {files[0]}')
        
    # Print file name to ensure expected dataset
    print(f'Attempting to open: {files[0]}')
    
    # Open file into xarray dataset
    bedmachine_original = xr.open_dataset(files[0], engine='netcdf4')
    print('Dataset successfully loaded')

except FileNotFoundError as e:
    print(f'File access error: {e}')
except ValueError as e:
    print(f'Search error: {e}')
except Exception as e:
    print(f'Unexpected error: {e}')

'''
Note: Search error: can only read bytes or file-like objects with engine='scipy' or 'h5netcdf' seems to indicate FileNotFoundError
'''

In [ ]:
# Open local copy of file if streaming fails
bedmachine_original = xr.open_dataset('/home/jovyan/temp/BedMachineAntarctica-v3.nc')
bedmachine_original

# Data analysis
Use Shreve (1972) equation to calculate hydropotential using bed elevations and ice-surface elevations (less firn depth to get ice mass).

In [ ]:
# Step 1: Compute hydropotential

# Use BedMachine bed topography for Zbed and surface topography (less firn) for Zsurf

# Define densities (rho) of ice and water
rho_ice = 917
rho_water = 997

# Calculate hydropotential using Shreve, 1972 equation
subglacial_hydropotential_kPa = (9.8 * ((rho_ice*(bedmachine_original['surface']-bedmachine_original['firn'])) + (rho_water-rho_ice)*bedmachine_original['bed'])) / 1e3

# Display xarray.Dataset metadata
subglacial_hydropotential_kPa

In [ ]:
print(subglacial_hydropotential_kPa.min(), subglacial_hydropotential_kPa.max())

In [ ]:
# Step 2: Mask to grounded ice only

# Create a mask where values == 2 (grounded ice)
grounded_ice_mask = (bedmachine_original['mask'] == 2)

# Apply the mask to subglacial_hydropotential_kPa
subglacial_hydropotential_kPa = subglacial_hydropotential_kPa.where(grounded_ice_mask)

# Display xarray.Dataset metadata to ensure edges have become nan's
subglacial_hydropotential_kPa

In [ ]:
def get_data_extent_with_buffer(da, buffer_percent=0.05):
    """
    Find the extent of non-NaN data and add a buffer.
    
    Parameters:
    -----------
    da : xarray.DataArray
        The data array to analyze
    buffer_percent : float
        Buffer as a percentage of the data range (default 5%)
    
    Returns:
    --------
    tuple : (x_min, x_max, y_min, y_max) with buffer applied
    """
    # Find where data is not NaN
    valid_data = ~np.isnan(da)
    
    # Get coordinates where we have valid data
    y_coords, x_coords = np.where(valid_data)
    
    if len(x_coords) == 0 or len(y_coords) == 0:
        print("Warning: No valid data found!")
        return None
    
    # Convert array indices to actual coordinate values
    x_values = da.coords[da.dims[1]].values  # assuming dims are [y, x]
    y_values = da.coords[da.dims[0]].values

    # Get the actual coordinate bounds where we have data
    x_data_min = x_values.min()
    x_data_max = x_values.max()
    y_data_min = y_values.min()
    y_data_max = y_values.max()
    
    # Calculate buffer
    x_range = x_data_max - x_data_min
    y_range = y_data_max - y_data_min
    
    x_buffer = x_range * buffer_percent
    y_buffer = y_range * buffer_percent
    
    # Apply buffer
    x_min = x_data_min - x_buffer
    x_max = x_data_max + x_buffer
    y_min = y_data_min - y_buffer
    y_max = y_data_max + y_buffer
    
    return x_min, x_max, y_min, y_max

# Get the data extent with 5% buffer
extent = get_data_extent_with_buffer(subglacial_hydropotential_kPa, buffer_percent=0.01)

if extent is not None:
    x_min, x_max, y_min, y_max = extent
    
    # Plot results and save figure
    fig, ax = plt.subplots(figsize=(10, 8))

    # Use actual data range for asymmetric colorbar
    data_min = float(subglacial_hydropotential_kPa.min())
    data_max = float(subglacial_hydropotential_kPa.max())

    # Create the plot
    im = subglacial_hydropotential_kPa.plot(
        ax=ax, 
        add_colorbar=False,
        cmap='viridis',
        vmin=data_min,
        vmax=data_max
    )

    # Set the axis limits to clip to data extent + buffer
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    
    # Format axes
    ax.set_aspect('equal')
    ax.set_axis_off()

    # Create colorbar axes with same width as plot
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('bottom', size='3%', pad=0.01)
    
    # Add colorbar with label
    cbar = plt.colorbar(im, cax=cax, orientation='horizontal')
    cbar.set_label('subglacial hydropotential [kPa]')
    cbar.ax.tick_params(labelrotation=45)
    
    plt.tight_layout()
    plt.show()

    plt.savefig('output/subglacial-hydropotential-Antarctica.png', bbox_inches='tight')

else:
    print('Could not determine data extent - plotting without clipping')
    # Fallback to original plot
    fig, ax = plt.subplots(figsize=(10, 8))
    im = subglacial_hydropotential_kPa.plot(ax=ax, add_colorbar=False, cmap='viridis')
    ax.set_aspect('equal')
    ax.set_axis_off()
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('bottom', size='3%', pad=0.01)
    cbar = plt.colorbar(im, cax=cax, orientation='horizontal')
    cbar.set_label('subglacial hydropotential [kPa]')
    cbar.ax.tick_params(labelrotation=45)
    plt.tight_layout()
    plt.show()

    plt.savefig('output/subglacial-hydropotential-Antarctica.png', bbox_inches='tight')

# Export hydropotental dataset

In [ ]:
# Add metadata
subglacial_hydropotential_kPa.attrs = {
    'long_name': 'hydropotential',
    'units': 'kPa',
    'ice_density' : 917.0,
    'ice_density_units': 'kg m-3',
    'freshwater_density' : 997.0,
    'freshwater_density_units': 'kg m-3',
}

# Convert to dataset for chucking with Zarr
ds = subglacial_hydropotential_kPa.to_dataset(name='hydropotential')

# Add metadata
ds.attrs = {
    'conventions': 'CF-1.8',
    'title': 'Antarctic subglacial hydropotential derived from BedMachine Antarctica',
    'description': 'Antarctic subglacial gridded hydropotential calculated using Shreve (1972) equation using BedMachine Antarctica v3 bed and ice topographies and firn air content (Morlighem et al., 2020; Morlighem, 2022).',
    'history': 'Created 2025-07-19',
    # 'identifier_product_DOI': 'doi:',  # Will be assigned upon first release with Zenodo
    # 'citation': 'Sauthoff, W. & Siegfried, M. R. (2025). Antarctic subglacial hydropotential [Data set]. Zenodo. https://doi.org/XX',
    'license': 'CC BY-SA 4.0',
    'region': 'Antarctica'
}

# # Copy mapping variable into new dataset
# ds['mapping'] = bedmachine_original['mapping']

# Copy mapping variable into new dataset as variable with zero dimensions
mapping_attrs = bedmachine_original['mapping'].attrs
ds['mapping'] = xr.DataArray(np.array(0, dtype=np.int32), attrs=mapping_attrs)

# Link the mapping variable to variable
ds['hydropotential'].attrs['grid_mapping'] = 'mapping'

# Copy global CRS-related attributes
ds.attrs['proj4'] = bedmachine_original.attrs['proj4']
ds.attrs['Projection'] = bedmachine_original.attrs['Projection']

# Add crs and crs_wkt
ds.attrs['crs'] = 'EPSG:3031'
crs = CRS.from_epsg(3031)
ds.attrs['crs_wkt'] = crs.to_wkt()

# View ds metadata
ds

## Export to chucked netcdf

In [ ]:
# Export with chunking + compression
ds.to_netcdf(
    'output/subglacial_hydropotential_Antarctica.nc',
    encoding={
        'hydropotential': {
            '_FillValue': -9999.0,
            'dtype': 'float32',
            'zlib': True,
            'complevel': 4,
            'chunksizes': (500, 500)
        }
    }
)

## Validate dataset after export
First check climate and forecast conventions, then importing exported file, viewing metadata and plotting data.

In [ ]:
!conda install -c conda-forge cfchecker --quiet

In [ ]:
!cfchecks output/subglacial_hydropotential_Antarctica.nc

In [ ]:
ds_imported = xr.open_dataset('output/subglacial_hydropotential_Antarctica.nc')
ds_imported

In [ ]:
ds_imported.info()

In [ ]:
plt.figure()
ds_imported['hydropotential'].plot(vmin=0, vmax=40e3)
plt.show()

In [ ]:
# Opened netcdf in Panoply to ensure compatibility

## Export to Zarr

In [ ]:
# Chunk for performance (tune based on size and use case)
ds = ds.chunk({'x': 500, 'y': 500})

# Write to a Zarr store
ds.to_zarr('output/subglacial_hydropotential_Antarctica.zarr', mode='w',  consolidated=True)

In [ ]:
# Zip zarr files for upload to Zenodo
shutil.make_archive(
    'output/subglacial_hydropotential_Antarctica.zarr',  # output path (no zip extension)
    'zip',
    'output/subglacial_hydropotential_Antarctica.zarr'   # source
)

In [ ]:
!rm -r 'output/subglacial_hydropotential_Antarctica.zarr'

## Validate dataset after export
Unzip zip file of Zarr store, then import data, view metadata, plot data.

In [ ]:
# Extract the zip file
with zipfile.ZipFile('output/subglacial_hydropotential_Antarctica.zarr.zip', 'r') as zip_ref:
    zip_ref.extractall('output/subglacial_hydropotential_Antarctica.zarr')

# Now open normally
ds_imported = xr.open_zarr('output/subglacial_hydropotential_Antarctica.zarr', consolidated=True)
ds_imported

In [ ]:
ds_imported.info()

In [ ]:
plt.figure()
ds_imported['hydropotential'].plot(vmin=0, vmax=40e3)
plt.show()

In [ ]:
# Remove Zarr store folder
!rm -r 'output/subglacial_hydropotential_Antarctica.zarr'

# Remove temporary files
Files are first downloaded locally to upload to Zenodo repo.

In [ ]:
!rm 'output/subglacial_hydropotential_Antarctica.nc'

In [ ]:
!rm 'output/subglacial_hydropotential_Antarctica.zarr.zip'

# References

Morlighem, M., Rignot, E., Binder, T., Blankenship, D., Drews, R., Eagles, G., et al. (2020). Deep glacial troughs and stabilizing ridges unveiled beneath the margins of the Antarctic ice sheet. Nature Geoscience, 13(2), 132–137. https://doi.org/10.1038/s41561-019-0510-8

Morlighem, M. (2022). MEaSUREs BedMachine Antarctica. (NSIDC-0756, Version 3). [Data Set]. Boulder, Colorado USA. NASA National Snow and Ice Data Center Distributed Active Archive Center. https://doi.org/10.5067/FPSU0V1MWUB6. Date Accessed 05-19-2025.

Sauthoff, W., & Siegfried, M. R. (2025). Antarctic subglacial hydropotential [Data set]. Zenodo. [DOI available after initial release]

Shreve, R. L. (1972). Movement of Water in Glaciers. _Journal of Glaciology_, 11(62), 205–214. https://doi.org/10.3189/S002214300002219X